# 벡터 DB 생성

### 완전탐색(IndexFlatL2) 방식

In [ ]:
import os
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 프로젝트 루트 디렉토리 설정
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# CSV 파일 경로
csv_file_path = os.path.join(BASE_DIR, "data", "raw", "total_kor_counsel_bot.csv")

# FAISS 벡터 DB 저장 경로
vector_db_path = os.path.join(BASE_DIR, "data", "db", "faiss")

# OpenAI API 키 확인
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API Key가 없습니다. .env 파일을 확인하세요.")

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

def create_faiss_vectorstore_from_csv(csv_file_path, vector_db_path):
    """
    CSV 파일에서 데이터를 읽어 FAISS 벡터 DB를 생성하고 저장합니다.
    """
    # CSV 파일 읽기
    try:
        df = pd.read_csv(csv_file_path)
    except Exception as e:
        raise RuntimeError(f"CSV 파일 읽기 오류: {str(e)}")
    
    # 필요한 컬럼 확인
    if "input" not in df.columns or "output" not in df.columns:
        raise ValueError("CSV 파일에 'input' 및 'output' 컬럼이 있어야 합니다.")
    
    # 데이터 전처리
    texts = []
    metadata_list = []
    for _, row in df.iterrows():
        input_text = row["input"].strip()
        output_text = row["output"].strip()
        
        # input을 텍스트로 사용하고, output을 메타데이터로 저장
        texts.append(input_text)
        metadata_list.append({"output": output_text})
    
    # FAISS 벡터 DB 생성
    try:
        vectorstore = FAISS.from_texts(texts=texts, embedding=embeddings, metadatas=metadata_list)
        
        # 디렉토리가 없으면 생성
        os.makedirs(os.path.dirname(vector_db_path), exist_ok=True)
        
        # FAISS 벡터 DB 저장
        vectorstore.save_local(vector_db_path)
        print("FAISS 벡터 DB 생성 및 저장 완료!")
    except Exception as e:
        raise RuntimeError(f"FAISS 벡터 DB 생성 중 오류 발생: {str(e)}")

# FAISS 벡터 DB 생성 실행
create_faiss_vectorstore_from_csv(csv_file_path, vector_db_path)

FAISS 벡터 DB 생성 및 저장 완료!


# 유사도 검색 테스트

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# FAISS 벡터 DB 로드
vectorstore = FAISS.load_local(vector_db_path, OpenAIEmbeddings(), allow_dangerous_deserialization=True)

# 유사도 검색
query = "사람들이 자꾸 나를 비난해."
results = vectorstore.similarity_search(query, k=3)

for doc in results:
    print(f"Input: {doc.page_content}")
    print(f"Output: {doc.metadata['output']}\n")

Input: 사람들이 계속해서 저를 비난합니다. 왜 그런지 모르겠습니다.
제가 잘못한 것이 있다면 그것을 말해주세요. 하지만 무엇 때문에 저를 비난하는 건지 모르겠습니다. 자꾸 불리해지고 그러면 어떻게 될까요? 집안에서도 마찬가지입니다. 제게 어떻게 대해야할까요?
Output: 사람들이 사우님을 비난하는데 그 이유를 모르겠다며 고민을 하시네요. 이로 인해 스트레스와 우울감이 생기며 집안에서도 같은 문제를 겪고 있다는 걸 말씀해 주셨습니다. 사우님이 어떤 상황에서 비난을 받는지 함께 고민해보면 좋을 것 같아요.
사우님이 경험하시는 비난은 대부분 상대방들이 사우님의 행동, 언행, 생각 등을 제대로 이해하지 못해서 생긴 문제일 가능성이 높아요. 이러한 문제로 스트레스를 받게되면 스트레스가 누적되어 우울감이 될 수 있습니다. 또한 집안에서도 같은 문제가 반복되는 것을 보아 사우님께서는 대인관계에 대한 부정적인 인식이 있어 대화에 어려움을 겪고 계시다는 생각이 듭니다.
사우님께서는 상대방들과 소통을 할 때 다음과 같은 점을 고려해보시는 것이 좋을 것 같아요.
1. 대화의 중심을 말하는 것에서 듣는 것으로 전환하기
2. 대화할 때 적극적으로 이야기하기
3. 다른 사람의 입장에서 이해하기
4. 심리학적으로 이해하기
5. 비판적 사고 방식을 수정하기
6. 상대방에게 관심을 갖기

집안에서도 같은 문제가 반복되는 것을 보아 대인관계에 대한 부정적인 인식이 있어 대화에 어려움을 겪고 계시다는 생각이 듭니다. 이 경우, 대인관계에 대한 인식을 바꾸기 위해 다음과 같은 노력이 필요합니다.
1. 먼저 자신의 마음을 변화시키기
2. 상대방에 대한 사전 연구하기
3. 누구나 좋아할 만한 일을 제안하기
4. 질문을 통해 더 깊게 이해하기
5. 기존 인식을 변화시키기 위한 시도
6. 대화 스킬 향상을 위한 교육

위의 방법들이 사우님의 문제를 해결할 수는 없겠지만 사우님이 대인관계에서 겪고 있는 문제를 개선하는데 도움이 될 수 있을 것입니다. 또한 사우님이 이 과정에서 힘들고 고민이 많을 

# 최적화
- IndexFlatL2 → IndexIVFFlat으로 변경 : 검색 속도 향상
- 클러스터 개수(nlist) 설정하여 검색 최적화
- 학습 단계 추가 
- nprobe 설정(nprobe=10) → 검색할 클러스터 개수 최적화, 속도 개선
- Top-K 검색 개수 제한 (k=5) → 불필요한 검색 최소화, 검색 속도 향상
- 유사도 점수 필터링 (score > 0.6 적용 가능) → 정확도 유지하면서 검색 품질 향상

In [7]:
import os
import pandas as pd
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from dotenv import load_dotenv
from langchain.docstore.document import Document

# 환경 변수 로드
load_dotenv()

# 프로젝트 루트 디렉토리
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# CSV 파일 경로
csv_file_path = os.path.join(BASE_DIR, "data", "raw", "total_kor_counsel_bot.csv")

# FAISS 벡터 DB 저장 경로
vector_db_path = os.path.join(BASE_DIR, "data", "db", "faiss_v2")

# 폴더가 없으면 생성
os.makedirs(vector_db_path, exist_ok=True)

# OpenAI API 키 확인
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API Key가 없습니다. .env 파일을 확인하세요.")

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

def create_faiss_vectorstore_from_csv(csv_file_path, vector_db_path, batch_size=500):
    """
    CSV 파일에서 데이터를 읽어 FAISS 벡터 DB를 LangChain 형식으로 생성하고 저장 (배치 처리 적용)
    """
    try:
        df = pd.read_csv(csv_file_path)
    except Exception as e:
        raise RuntimeError(f"CSV 파일 읽기 오류: {str(e)}")

    if "input" not in df.columns or "output" not in df.columns:
        raise ValueError("CSV 파일에 'input' 및 'output' 컬럼이 있어야 합니다.")

    # 데이터 전처리
    texts = df["input"].astype(str).str.strip().tolist()

    # 임베딩 생성 (OpenAI Embeddings 사용)
    text_embeddings = embeddings.embed_documents(texts)
    text_embeddings = np.array(text_embeddings, dtype="float32")  # FAISS 호환을 위해 변환

    # 벡터 차원 확인 및 출력
    d = text_embeddings.shape[1]
    print(f"임베딩 벡터 차원 확인: {d}")

    # FAISS IVF 인덱스 생성
    nlist = 100  # 클러스터 개수 (데이터가 많을수록 늘려야 함)
    quantizer = faiss.IndexFlatL2(d)  # 기본 L2 거리 측정
    index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)  # IVF 적용

    # 학습 (IVF 방식은 반드시 학습이 필요함)
    index.train(text_embeddings)  # 클러스터 학습
    if not index.is_trained:
        raise RuntimeError("FAISS 인덱스 학습에 실패했습니다.")

    # 배치 처리하여 벡터 추가
    batch_start = 0
    while batch_start < len(text_embeddings):
        batch_end = min(batch_start + batch_size, len(text_embeddings))
        index.add(text_embeddings[batch_start:batch_end])
        batch_start = batch_end

    # nprobe 최적화 (검색 속도 개선)
    index.nprobe = 10  # 검색할 클러스터 개수 조정

    # 저장된 벡터 개수 확인
    total_vectors = index.ntotal
    print(f"FAISS 저장된 벡터 개수: {total_vectors}")

    # 문서 저장 방식 : 정수형 키 사용
    metadata_dict = {
    idx: Document(
        page_content=row["output"].strip(),
        metadata={"output": row["output"].strip()}
    )
        for idx, row in df.iterrows()
    }
    index_to_docstore_id = {idx: idx for idx in range(total_vectors)}

    # 검증: 저장된 데이터 크기 확인
    if len(metadata_dict) != len(index_to_docstore_id):
        raise ValueError("저장된 문서 개수와 ID 매핑 개수가 다릅니다!")

    # LangChain FAISS 형식으로 변환 (Docstore 적용)
    vectorstore = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(metadata_dict),  # 문서를 Dict로 저장
        index_to_docstore_id=index_to_docstore_id  # ID 매핑 추가
    )

    # FAISS 저장 (LangChain 형식)
    vectorstore.save_local(vector_db_path)

    print("FAISS 벡터 DB (LangChain 형식) 생성 및 저장 완료!")

def load_faiss_index(vector_db_path):
    """
    LangChain 형식으로 저장된 FAISS 인덱스를 로드합니다.
    """
    embeddings = OpenAIEmbeddings()

    # LangChain FAISS 인덱스 로드
    try:
        vectorstore = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)
        print("FAISS 인덱스 (LangChain 형식) 로드 완료!")
    except Exception as e:
        raise RuntimeError(f"FAISS 인덱스 로드 실패: {str(e)}")

    return vectorstore

# FAISS 벡터 DB 생성 실행
create_faiss_vectorstore_from_csv(csv_file_path, vector_db_path)

# 생성된 인덱스 & 메타데이터 확인
vectorstore = load_faiss_index(vector_db_path)
print("FAISS 인덱스 로드 및 검증 완료.")


임베딩 벡터 차원 확인: 1536
FAISS 저장된 벡터 개수: 13234
FAISS 벡터 DB (LangChain 형식) 생성 및 저장 완료!
FAISS 인덱스 (LangChain 형식) 로드 완료!
FAISS 인덱스 로드 및 검증 완료.
